In [52]:
import numpy as np
import pandas as pd
import yfinance as yf

In [53]:
# SPX_data = yf.download('SPX',start='2006-01-01',end='2006-12-31')

In [54]:
# SPX_data.to_csv('SPX_1year_data_from_jan2006.csv')

In [55]:
df = pd.read_csv('datasets/SPX_1year_data_from_jan2006.csv')

In [56]:
df['Date']=df['Date'].str.replace('-','').astype('int')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,20060103,1.18671,1.18671,1.06179,1.06179,1.06179,19933
1,20060104,1.06179,1.12425,1.06179,1.12425,1.12425,2562
2,20060105,1.06179,1.06179,1.06179,1.06179,1.06179,640
3,20060106,0.99934,1.18671,0.99934,1.18671,1.18671,1921
4,20060109,1.18671,1.18671,1.06179,1.06179,1.06179,3602


In [57]:
df.set_index('Date',inplace=True)
df.sort_index(inplace =True)

In [62]:

dailyret = df.pct_change()

marketDailyret = dailyret.mean(axis=1)
marketDailyret.replace([-np.inf,np.inf],np.nan,inplace=True)
marketDailyret.fillna(marketDailyret.mean())
marketDailyret


Date
20060103         NaN
20060104   -0.151953
20060105   -0.152811
20060106    0.372814
20060109    0.152420
              ...   
20061222   -0.108666
20061226   -0.171604
20061227         NaN
20061228   -0.074101
20061229    0.962020
Length: 251, dtype: float64

In [63]:
weights = -(np.array(dailyret)-np.array(marketDailyret).reshape((dailyret.shape[0],1)))

In [64]:
wtsum = np.nansum(abs(weights),axis =1)

In [65]:
weights[wtsum==0,]=0

In [67]:
wtsum[wtsum==0]=1

In [68]:
weights = weights/ wtsum.reshape((dailyret.shape[0],1))

In [70]:
dailypnl = np.nansum(np.array(pd.DataFrame(weights).shift())*np.array(dailyret),axis=1)

In [72]:
dailypnl = dailypnl[np.logical_and(df.index>=20060101,df.index<= 20061231)]

In [75]:
mean_val = np.mean(dailypnl[np.isfinite(dailypnl)])

In [76]:
dailypnl = np.where(np.isinf(dailypnl),mean_val,dailypnl)

In [77]:
sharpeRatio = np.sqrt(252)*np.mean(dailypnl)/np.std(dailypnl)

In [79]:
sharpeRatio

1.732260880548924

In [80]:
onewaycost = 0.0005

In [85]:
weights=weights[np.logical_and(df.index >= 20060101,
df.index <= 20061231)]

In [86]:
dailypnlminustcost = dailypnl-(np.nansum(abs(weights-np.array(pd.DataFrame(weights).shift())),axis=1)*onewaycost)

In [87]:
sharpeRatioMinusTcost = np.sqrt(252)*np.mean(dailypnlminustcost)/np.std(dailypnlminustcost)

In [88]:
sharpeRatioMinusTcost

1.7314833806968557